In [62]:
import pymysql
import os
import requests
import re
import json
from bs4 import BeautifulSoup as bs

In [63]:
def getSoup(url, session):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.36',
        }
    response = session.get(url, headers=headers, allow_redirects=True)
    print(response)
    html = response.text
    soup = bs(html)
    print(session.cookies)
    return soup

In [64]:
def CreateTable():
    connection = pymysql.connect(host='localhost',
                             user='root',
                             password='root',
                             db='stocks',
                             charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor)
    
    creatSql = """create table if not exists total_stocks(
                `id` INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
                `code` INT NOT NULL UNIQUE,
                `name` VARCHAR(20) NOT NULL,
                `price` FLOAT NOT NULL,
                `up_down_rate` FLOAT NOT NULL,
                `up_down` FLOAT NOT NULL,
                `up_rate` FLOAT NOT NULL,
                `change_rate` FLOAT NOT NULL,
                `quantity_rate` FLOAT NOT NULL,
                `amplitude_rate` FLOAT NOT NULL,
                `transaction` VARCHAR(20) NOT NULL,
                `circulating` VARCHAR(20) NOT NULL,
                `circulation_market` VARCHAR(20) NOT NULL,
                `pe_ratio` VARCHAR(20) NOT NULL
                );"""
    
    try:
        with connection.cursor() as cursor:
            cursor.execute(creatSql)
        connection.commit()
    finally:
        connection.close()


In [65]:
def writeData(trlist):
    connection = pymysql.connect(host='localhost',
                             user='root',
                             password='root',
                             db='stocks',
                             charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor)
    sql = """
            INSERT total_stocks (`code`, `name`, `price`, `up_down_rate`, `up_down`, `up_rate`,
            `change_rate`, `quantity_rate`, `amplitude_rate`, `transaction`, `circulating`,
            `circulation_market`, `pe_ratio`)
            VALUES
            (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
           """
    try:
        with connection.cursor() as cursor:
            # execute执行一条，executemany批量插入
            cursor.executemany(sql, trlist)

        connection.commit()
    finally:
        connection.close()

In [66]:
def is_number(string):
    pattern = re.compile(r'^[-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?$')
    return bool(pattern.match(string))

In [67]:
def main():
    CreateTable()
    trlist = []
    session = requests.Session()
    mainUrl = 'http://q.10jqka.com.cn/'
    mainSoup = getSoup(mainUrl, session)
    for i in range(1, 264):
        url = f'http://q.10jqka.com.cn/index/index/board/all/field/zdf/order/desc/page/{i}/ajax/1/'
        soup = getSoup(url, session)
        table = soup.find(name='table', attrs={'class': 'm-table m-pager-table'})
        trs = table.tbody.findAll(name='tr')
        for tr in trs:
            tds = tr.findAll(name='td')
            tdlist = []
            for i, td in enumerate(tds):
                if is_number(td.text) and i != 13:
                    if '.' in td.text:
                        text = float(td.text)
                    else:
                        text = int(td.text)
                    tdlist.append(text)
                else:
                    tdlist.append(td.text)
            trlist.append(tdlist[1: -1])
    print(trlist)
    writeData(trlist)
          

In [68]:
if __name__ == "__main__":
    main()

KeyError: 'name=0, domain=None, path=None'